# Готовим данные, на которых будут строиться регрессии

- Джоиним сырые данные о торговле на регрессоры
- Джоиним фиксированные торговые эффекты на регрессоры

Рисуем scatterplot, чтобы выбрать правильную спецификацию модели

In [1]:
import polars as pl
import plotly.express as  px


In [2]:
regressors = pl.read_parquet('../preprocessed_data/regressors.parquet.gzip').with_columns(pl.col("t").cast(pl.Utf8).alias("year"))
sql = pl.SQLContext(register_globals=True)
sql.execute('''select * from regressors limit 2;''').collect()

country,t,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated,country:temp,t:temp,average_surface_temperature,country:median_age_table,t:median_age_table,median_age,country:SARS_table,SARS_total_cases,country:density_and_mortality,t:density_and_mortality,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization,country:household_size,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members,year
str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,str
"""SLV""",2020,19.625912,0.570608,null,2.426229,1.434426,1.306011,2.297814,0.833333,1.800546,1.295082,2.70765,0.751366,1.445355,4.871233e6,0.0,1.879781,0.808743,0.631148,489863.0,0.0,2.196721,0.0,2.046448,63.594536,53.958168,53.822815,54.918034,0.0,0.0,"""SLV""",2020,26.48445,"""SLV""",2020,25.691,null,null,"""SLV""",2020,300.901207,52.477556,null,null,null,null,null,124.344002,293.265015,11.0,24.148672,17.73575,"""SLV""",3.92,8.46,36.009998,37.84,"""2020"""
"""ASM""",2022,467.620027,1.926,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ASM""",2022,26.570254,"""ASM""",2022,27.927,null,null,"""ASM""",2022,241.71,49.464234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""2022"""


## Сырые данные о торговле

In [3]:
trade = pl.read_parquet('../preprocessed_data/trade_hs2.parquet.gzip')
sql.register('trade', trade)
# sql.execute('''select * from trade limit 2;''').collect()
sql.execute('''create table trade_x_covid as
    select *
    from trade tr
    inner join regressors r_i 
        on tr.i = r_i.country
        and tr.t = r_i.t        
    inner join regressors r_j 
        on tr.j = r_j.country
        and tr.t = r_j.t                               
;''')
sql.execute('''select * from trade_x_covid limit 2;''').collect()

t,i,j,category,v,q,v19,q19,country,t:r_i,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,…,H6M_Facial_Coverings:r_j,H7_Vaccination_policy:r_j,H8M_Protection_of_elderly_people:r_j,StringencyIndex_Average:r_j,GovernmentResponseIndex_Average:r_j,ContainmentHealthIndex_Average:r_j,EconomicSupportIndex:r_j,MajorityVaccinated:r_j,PopulationVaccinated:r_j,country:temp:r_j,t:temp:r_j,average_surface_temperature:r_j,country:median_age_table:r_j,t:median_age_table:r_j,median_age:r_j,country:SARS_table:r_j,SARS_total_cases:r_j,country:density_and_mortality:r_j,t:density_and_mortality:r_j,population_density:r_j,female_ratio:r_j,maternal_mortality:r_j,cvd_mortality:r_j,pollution_mortality:r_j,poisoning_mortality:r_j,hygiene_mortality:r_j,female_mortality:r_j,male_mortality:r_j,infant_mortality:r_j,largest_city_share:r_j,urbanization:r_j,country:household_size:r_j,avg_hh_size:r_j,share_1_members:r_j,share_2_3_members:r_j,share_4_5_members:r_j,year:r_j
i16,str,str,str,f32,f32,f64,f64,str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,str
2021,"""SVK""","""RUS""","""12""",37.881001,18.0,310.385,104.0,"""SVK""",2021,329.429178,7.320849,null,1.961644,2.115068,2.0,3.764384,0.0,0.936986,0.684932,1.545205,2.0,2.0,2.2361996e7,0.0,1.912329,2.953425,1.350685,0.0,0.0,3.065753,4.172603,2.106849,52.952766,66.085045,62.081944,94.109589,…,2.315068,4.80274,1.421918,46.464878,54.567123,55.111725,50.753426,0.0,16.470549,"""RUS""",2021,-3.462058,"""RUS""",2021,38.855999,"""RUS""",1,"""RUS""",2021,8.838488,53.545736,null,null,null,null,null,null,null,4.1,11.610478,23.469076,"""RUS""",2.58,25.690001,51.040001,19.84,"""2021"""
2021,"""JAM""","""FRA""","""04""",7.27,4.502,0.16,0.022,"""JAM""",2021,76.880959,2.092082,null,1.446575,2.227397,1.917808,3.586301,1.041096,2.128767,1.986301,2.427397,0.460274,0.460274,0.0,5479.452148,1.547945,2.90137,2.0,4.7945204e7,113009.179688,3.095891,3.336986,2.336986,73.649231,70.476463,77.257805,23.013699,…,3.2,4.186301,2.435616,52.678246,58.881783,61.343643,41.643837,0.40274,34.622875,"""FRA""",2021,13.359396,"""FRA""",2021,41.389999,"""FRA""",7,"""FRA""",2021,123.900911,51.551507,null,null,null,null,null,46.098,93.640999,3.3,20.100105,22.838049,null,null,null,null,null,"""2021"""


In [4]:
sql.execute('''select * from trade_x_covid;''').collect().write_parquet('../processed_data/trade_hs2_x_covid.parquet.gzip')

## Фиксированные эффекты

### Экспорт

In [5]:
trade = pl.read_parquet('../preprocessed_data/trade_hs2_cat_interact_value_plain_export_fe.parquet.gzip')
sql.register('trade', trade)
# sql.execute('''select * from trade limit 2;''').collect()
sql.execute('''create table trade_x_covid as
    select *
    from trade t
    inner join regressors r 
    on t.country = r.country
    and t.year = r.t                       
;''')
sql.execute('''select * from trade_x_covid limit 2;''').collect()

year,country,category,value,country:r,t,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated,country:temp,t:temp,average_surface_temperature,country:median_age_table,t:median_age_table,median_age,country:SARS_table,SARS_total_cases,country:density_and_mortality,t:density_and_mortality,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization,country:household_size,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members,year:r
i32,str,str,f64,str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,str
2020,"""ABW""","""03""",5.267693,"""ABW""",2020,133.980138,1.204475,null,1.385246,1.191257,1.4153,2.081967,0.0,1.103825,0.614754,2.051913,1.051913,1.448087,0.0,0.0,1.579235,0.811475,0.786885,0.0,0.0,1.054645,0.0,2.150273,45.165165,42.321583,40.779865,53.107922,0.0,0.0,null,null,null,"""ABW""",2020,40.398998,null,null,"""ABW""",2020,603.261111,52.844507,null,null,null,null,null,59.681999,124.098999,null,null,null,"""ABW""",2.89,21.370001,46.77,25.59,"""2020"""
2020,"""ABW""","""07""",-1.537117,"""ABW""",2020,133.980138,1.204475,null,1.385246,1.191257,1.4153,2.081967,0.0,1.103825,0.614754,2.051913,1.051913,1.448087,0.0,0.0,1.579235,0.811475,0.786885,0.0,0.0,1.054645,0.0,2.150273,45.165165,42.321583,40.779865,53.107922,0.0,0.0,null,null,null,"""ABW""",2020,40.398998,null,null,"""ABW""",2020,603.261111,52.844507,null,null,null,null,null,59.681999,124.098999,null,null,null,"""ABW""",2.89,21.370001,46.77,25.59,"""2020"""


In [6]:
sql.execute('''select * from trade_x_covid;''').collect().write_parquet('../processed_data/trade_hs2_export_fe_x_covid.parquet.gzip')

### Импорт

In [7]:
trade = pl.read_parquet('../preprocessed_data/trade_hs2_cat_interact_value_plain_import_fe.parquet.gzip')
sql.register('trade', trade)
# sql.execute('''select * from trade limit 2;''').collect()
sql.execute('''create table trade_x_covid as
    select *
    from trade t
    inner join regressors r 
    on t.country = r.country
    and t.year = r.t                       
;''')
sql.execute('''select * from trade_x_covid limit 2;''').collect()

year,country,category,value,country:r,t,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated,country:temp,t:temp,average_surface_temperature,country:median_age_table,t:median_age_table,median_age,country:SARS_table,SARS_total_cases,country:density_and_mortality,t:density_and_mortality,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization,country:household_size,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members,year:r
i32,str,str,f64,str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,str
2020,"""ABW""","""01""",-1.876429,"""ABW""",2020,133.980138,1.204475,null,1.385246,1.191257,1.4153,2.081967,0.0,1.103825,0.614754,2.051913,1.051913,1.448087,0.0,0.0,1.579235,0.811475,0.786885,0.0,0.0,1.054645,0.0,2.150273,45.165165,42.321583,40.779865,53.107922,0.0,0.0,null,null,null,"""ABW""",2020,40.398998,null,null,"""ABW""",2020,603.261111,52.844507,null,null,null,null,null,59.681999,124.098999,null,null,null,"""ABW""",2.89,21.370001,46.77,25.59,"""2020"""
2020,"""ABW""","""02""",-1.192187,"""ABW""",2020,133.980138,1.204475,null,1.385246,1.191257,1.4153,2.081967,0.0,1.103825,0.614754,2.051913,1.051913,1.448087,0.0,0.0,1.579235,0.811475,0.786885,0.0,0.0,1.054645,0.0,2.150273,45.165165,42.321583,40.779865,53.107922,0.0,0.0,null,null,null,"""ABW""",2020,40.398998,null,null,"""ABW""",2020,603.261111,52.844507,null,null,null,null,null,59.681999,124.098999,null,null,null,"""ABW""",2.89,21.370001,46.77,25.59,"""2020"""


In [8]:
sql.execute('''select * from trade_x_covid;''').collect().write_parquet('../processed_data/trade_hs2_import_fe_x_covid.parquet.gzip')